In [1]:
!pip install -q espnet==0.10.6 pyopenjtalk==0.2 pypinyin==0.44.0 parallel_wavegan==0.5.4 gdown==4.4.0 espnet_model_zoo pydub

In [4]:
from pymongo import MongoClient
import pandas as pd
from espnet2.bin.tts_inference import Text2Speech
from espnet2.utils.types import str_or_none
import time
import torch
from IPython.display import display, Audio
import soundfile as sf
import pydub

Default Voice: VITS

In [5]:
tag="espnet/kan-bayashi_ljspeech_vits"

text2speech = Text2Speech.from_pretrained(
    model_tag=str_or_none(tag),
    #device="cuda",
    # Only for FastSpeech & FastSpeech2 & VITS
    speed_control_alpha=1.0,
    # Only for VITS
    noise_scale=0.333,
    noise_scale_dur=0.333,
)

In [15]:
i='1'
print("Enter text:")
x=str(input())
with torch.no_grad():
    start = time.time()
    wav = text2speech(x)["wav"]
rtf = (time.time() - start) / (len(wav) / text2speech.fs)
print(f"RTF = {rtf:5f}")

display(Audio(wav.view(-1).cpu().numpy(), rate=text2speech.fs))
sf.write("speech"+i+".wav", wav.to('cpu').numpy(), 22050)

sound = pydub.AudioSegment.from_wav("/content/speech"+i+".wav")
sound.export("voice"+i+".mp3", format="mp3")

Enter text:
The data sources here include access to different API, OBD data, external annotated image data bases. Accessing API data requires permission from sources. Data gathering is to be done from social media platforms using web scraping techniques.
RTF = 1.129195


<_io.BufferedRandom name='voice1.mp3'>

Advanced voice options : VITS-XVECTOR

In [93]:
tag = 'kan-bayashi/libritts_xvector_vits'

text2speech = Text2Speech.from_pretrained(
    model_tag=str_or_none(tag),
    #device="cuda",
    # Only for FastSpeech & FastSpeech2 & VITS
    speed_control_alpha=1.15,
    # Only for VITS
    noise_scale=0.333,
    noise_scale_dur=0.333,
)

In [88]:
import glob
import os
import numpy as np
import kaldiio

# Get model directory path
from espnet_model_zoo.downloader import ModelDownloader
d = ModelDownloader()
model_dir = os.path.dirname(d.download_and_unpack(tag)["train_config"])

# X-vector selection
spembs = None
if text2speech.use_spembs:
    xvector_ark = [p for p in glob.glob(f"{model_dir}/../../dump/**/spk_xvector.ark", recursive=True) if "tr" in p][0]
    xvectors = {k: v for k, v in kaldiio.load_ark(xvector_ark)}
    spks = list(xvectors.keys())

    # randomly select speaker
    random_spk_idx = np.random.randint(0, len(spks))
    spk = spks[random_spk_idx]
    spembs = xvectors[spk]
    print(f"selected spk: {spk}")

# Speaker ID selection
sids = None
if text2speech.use_sids:
    spk2sid = glob.glob(f"{model_dir}/../../dump/**/spk2sid", recursive=True)[0]
    with open(spk2sid) as f:
        lines = [line.strip() for line in f.readlines()]
    sid2spk = {int(line.split()[1]): line.split()[0] for line in lines}

    # randomly select speaker
    sids = np.array(np.random.randint(1, len(sid2spk)))
    spk = sid2spk[int(sids)]
    print(f"selected spk: {spk}")

# Reference speech selection for GST
speech = None
if text2speech.use_speech:
    # you can change here to load your own reference speech
    # e.g.
    # import soundfile as sf
    # speech, fs = sf.read("/path/to/reference.wav")
    # speech = torch.from_numpy(speech).float()
    speech = torch.randn(50000,) * 0.01

selected spk: 8797_294123


In [82]:
#spks.index('7285_72200')
#spks.index('4160_11549')

#2910_131096

1123

In [94]:
i='2'
print("Enter text:")
x=str(input())
with torch.no_grad():
    start = time.time()
    wav = text2speech(x, speech=speech, spembs=spembs, sids=sids)["wav"]

rtf = (time.time() - start) / (len(wav) / text2speech.fs)
print(f"RTF = {rtf:5f}")

display(Audio(wav.view(-1).cpu().numpy(), rate=text2speech.fs))
sf.write("speech"+i+".wav", wav.to('cpu').numpy(), 22050)

sound = pydub.AudioSegment.from_wav("/content/speech"+i+".wav")
sound.export("voice"+i+".mp3", format="mp3")

Enter text:
The data sources here include access to different API, OBD data, external annotated image data bases. Accessing API data requires permission from sources. Data gathering is to be done from social media platforms using web scraping techniques.
RTF = 1.215803


<_io.BufferedRandom name='voice2.mp3'>

Access data from csv

In [ ]:
#from pymongo import MongoClient
import pandas as pd
from espnet2.bin.tts_inference import Text2Speech
from espnet2.utils.types import str_or_none
import time
import torch
from IPython.display import display, Audio
import soundfile as sf
import pydub

tag="espnet/kan-bayashi_ljspeech_vits"

text2speech = Text2Speech.from_pretrained(
    model_tag=str_or_none(tag),
    #device="cuda",
    # Only for FastSpeech & FastSpeech2 & VITS
    speed_control_alpha=1.0,
    # Only for VITS
    noise_scale=0.333,
    noise_scale_dur=0.333,
)

def text_to_speech(text_col_name):
  print("please enter the url of the document")
  data=pd.read_csv(str(input()))
  text=data[text_col_name].to_list()
  x=None
  ops={}
  for i in range(len(text)):
    with torch.no_grad():
      start = time.time()
      wav = text2speech(text[i])["wav"]
      rtf = (time.time() - start) / (len(wav) / text2speech.fs)
      print(f"RTF = {rtf:5f}")
      display(Audio(wav.view(-1).cpu().numpy(), rate=text2speech.fs))
      sf.write("speech"+str(i)+".wav", wav.to('cpu').detach().numpy()[0], 22050)
      sound = pydub.AudioSegment.from_wav("/content/speech"+str(i)+".wav")
      sound.export("voice"+str(i)+".mp3", format="mp3")